In [5]:
import matplotlib.pyplot as plt
from pyod.models.pca import PCA as PCA
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
from pyod.utils.utility import standardizer
from sklearn.preprocessing import StandardScaler, RobustScaler